# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [84]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from collections import Counter 

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [4]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [15]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [6]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [7]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [8]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [9]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [10]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [11]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [12]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [13]:
product_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'euclidean'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)
product_dist

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.881127,0.882219,0.884604,0.890449,0.879859,0.885648,0.880852,0.886501,0.890573,...,0.884117,0.888124,0.886567,0.879578,0.875696,0.878427,0.886163,0.886564,0.887956,0.878223
"Appetizer - Mini Egg Roll, Shrimp",0.881127,1.000000,0.883877,0.880604,0.887930,0.881577,0.885241,0.890196,0.888646,0.883484,...,0.887569,0.887831,0.888033,0.881019,0.878720,0.871915,0.888702,0.890837,0.887661,0.882616
Appetizer - Mushroom Tart,0.882219,0.883877,1.000000,0.884708,0.883266,0.884124,0.886495,0.886072,0.887707,0.881019,...,0.886796,0.885968,0.885056,0.887445,0.882789,0.879286,0.890062,0.889960,0.889397,0.881077
Appetizer - Sausage Rolls,0.884604,0.880604,0.884708,1.000000,0.885025,0.885020,0.881992,0.883090,0.880745,0.881226,...,0.882793,0.886507,0.884984,0.886515,0.877195,0.868976,0.885470,0.884688,0.883661,0.877701
Apricots - Dried,0.890449,0.887930,0.883266,0.885025,1.000000,0.886792,0.887542,0.889058,0.886234,0.887162,...,0.886223,0.889001,0.890087,0.887478,0.878571,0.872542,0.892076,0.891209,0.889794,0.886477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.878427,0.871915,0.879286,0.868976,0.872542,0.875465,0.876566,0.877387,0.877911,0.876792,...,0.882272,0.877286,0.874760,0.872730,0.869942,1.000000,0.876589,0.875700,0.877148,0.871735
Yoghurt Tubes,0.886163,0.888702,0.890062,0.885470,0.892076,0.887069,0.893496,0.889616,0.888383,0.888293,...,0.891366,0.892520,0.890222,0.884474,0.885830,0.876589,1.000000,0.889996,0.891179,0.885087
"Yogurt - Blueberry, 175 Gr",0.886564,0.890837,0.889960,0.884688,0.891209,0.891202,0.890707,0.889430,0.891434,0.886262,...,0.890363,0.892818,0.890805,0.885548,0.880785,0.875700,0.889996,1.000000,0.890230,0.886320


### Step 2: Get the products purchased for a specific customer of your choice.

In [122]:
prod=prod_cust_pivot[33].reset_index()

In [126]:
products=list(prod[prod[33]!=0]['ProductName'])

In [128]:
len(products)

59

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [129]:
similar_prod=[]
for p in products: 
    similar_prod.append(list(product_dist[p].sort_values(ascending=False)[1:].head().index))

In [130]:
flat_list = [item for sublist in similar_prod for item in sublist]

In [131]:
for a in flat_list:
    if a in products:
        flat_list.remove(a)       

In [132]:
len(flat_list)

260

In [133]:
flat_list[0]

'Table Cloth 81x81 White'

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

1st way 

In [134]:
counting=[]
what=[]
for a in flat_list:
    counting.append(flat_list.count(a))
    what.append(a)

In [135]:
dff=pd.DataFrame(set(list(zip(what,counting))),columns=['what','counting'])

In [136]:
dfreal = dff.sort_values(by='counting', ascending=False)

In [137]:
dfrealreal=dfreal[:5]
dfrealrealreal= list(dfrealreal['what'])

In [138]:
dfrealrealreal

['Sun - Dried Tomatoes',
 'Spinach - Baby',
 'Beef - Montreal Smoked Brisket',
 'Rosemary - Primerba, Paste',
 'Cookies - Assorted']

2nd way 

In [139]:
times={}
for f in flat_list:
    times[f]=times.get(f,0)+1

In [140]:
k = Counter(times) 

In [141]:
high = k.most_common(5)

In [142]:
top5=[]
for h in high:
    top5.append(h[0])
    

In [143]:
top5

['Sun - Dried Tomatoes',
 'Spinach - Baby',
 'Beef - Montreal Smoked Brisket',
 'Cookies - Assorted',
 'Rosemary - Primerba, Paste']

In [82]:
'''
inverse = [(value, key) for key, value in times.items()]
print (max(inverse)[1])
'''

Rosemary - Primerba, Paste


### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [144]:
recommen={}
cust=set(list(data['CustomerID']))
for c in cust:
    prod=prod_cust_pivot[c].reset_index()
    products=list(prod[prod[c]!=0]['ProductName'])
    similar_prod=[]
    for p in products: 
        similar_prod.append(list(product_dist[p].sort_values(ascending=False)[1:].head().index))
    flat_list = [item for sublist in similar_prod for item in sublist]
    for a in flat_list:
        if a in products:
            flat_list.remove(a) 
    times={}
    for f in flat_list:
        times[f]=times.get(f,0)+1
    k = Counter(times)
    high = k.most_common(5)
    top5=[]
    for h in high:
        top5.append(h[0]) 
    recommen[c]=top5

{83973: ['Rosemary - Primerba, Paste',
  'Sun - Dried Tomatoes',
  'Cookies - Assorted',
  'Chocolate - Semi Sweet, Calets',
  'Muffin - Zero Transfat'],
 59399: ['Rosemary - Primerba, Paste',
  'Sun - Dried Tomatoes',
  'Oil - Shortening - All - Purpose',
  'Spinach - Baby',
  'Yogurt - Blueberry, 175 Gr'],
 92168: ['Sun - Dried Tomatoes',
  'Spinach - Baby',
  'Yogurt - Blueberry, 175 Gr',
  'Rosemary - Primerba, Paste',
  'Towels - Paper / Kraft'],
 49159: ['Rosemary - Primerba, Paste',
  'Spinach - Baby',
  'Towels - Paper / Kraft',
  'Sun - Dried Tomatoes',
  'Cookies - Assorted'],
 18441: ['V8 - Berry Blend',
  'Rosemary - Primerba, Paste',
  'Oil - Shortening - All - Purpose',
  'Sun - Dried Tomatoes',
  'Cod - Black Whole Fillet'],
 22536: ['Rosemary - Primerba, Paste',
  'Cookies - Assorted',
  'Spinach - Baby',
  'French Pastry - Mini Chocolate',
  'Rice - Jasmine Sented'],
 86028: ['Spinach - Baby',
  'Oil - Shortening - All - Purpose',
  'V8 - Berry Blend',
  'Rosemary - Pr

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [146]:
df1=pd.DataFrame(recommen)
df2=df1.T
df2

,0,1,2,3,4
83973,"Rosemary - Primerba, Paste",Sun - Dried Tomatoes,Cookies - Assorted,"Chocolate - Semi Sweet, Calets",Muffin - Zero Transfat
59399,"Rosemary - Primerba, Paste",Sun - Dried Tomatoes,Oil - Shortening - All - Purpose,Spinach - Baby,"Yogurt - Blueberry, 175 Gr"
92168,Sun - Dried Tomatoes,Spinach - Baby,"Yogurt - Blueberry, 175 Gr","Rosemary - Primerba, Paste",Towels - Paper / Kraft
49159,"Rosemary - Primerba, Paste",Spinach - Baby,Towels - Paper / Kraft,Sun - Dried Tomatoes,Cookies - Assorted
18441,V8 - Berry Blend,"Rosemary - Primerba, Paste",Oil - Shortening - All - Purpose,Sun - Dried Tomatoes,Cod - Black Whole Fillet
...,...,...,...,...,...
77814,"Rosemary - Primerba, Paste",Oil - Shortening - All - Purpose,Sun - Dried Tomatoes,"Hickory Smoke, Liquid",Cookies - Assorted
32753,"Rosemary - Primerba, Paste",Oil - Shortening - All - Purpose,Cookies - Assorted,Sun - Dried Tomatoes,V8 - Berry Blend
20476,Oil - Shortening - All - Purpose,"Hickory Smoke, Liquid","Rosemary - Primerba, Paste","Chocolate - Semi Sweet, Calets",Spinach - Baby
67582,Oil - Shortening - All - Purpose,Spinach - Baby,"Rosemary - Primerba, Paste",Muffin - Carrot Individual Wrap,"Yogurt - Blueberry, 175 Gr"


## BONUS: Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [161]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

In [164]:
prod_cust_pivot["new_customer"] = pd.Series(new_customer).fillna(0)
prod_cust_pivot1= prod_cust_pivot.fillna(0)

In [165]:
cus_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot1.T, 'euclidean'))), index=prod_cust_pivot1.columns, columns=prod_cust_pivot1.columns)
cus_dist

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,new_customer
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333,0.067975
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335,0.063707
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333,0.067640
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339,0.063985
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349,0.070847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381,0.004585
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266,0.004365
98185,0.004167,0.004166,0.004179,0.004175,0.004177,0.004167,0.004168,0.004176,0.004172,0.004161,...,0.003162,0.003255,0.003406,0.003172,0.003255,0.003345,0.003277,1.000000,0.003288,0.004155


### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [166]:
simil_cust = list(cus_dist['new_customer'].sort_values(ascending=False)[1:].head().index)
simil_cust

[1072, 412, 1534, 2503, 3885]

In [167]:
simil_cust_prod = customer_products[customer_products['CustomerID'].isin(simil_cust)]
groupe = simil_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
rank_products = groupe.sort_values('Quantity', ascending=False).reset_index()
rank_products

,ProductName,Quantity
0,"Sole - Dover, Whole, Fresh",4
1,Knife Plastic - White,4
2,Spinach - Baby,3
3,Muffin - Carrot Individual Wrap,3
4,Beef - Short Loin,3
...,...,...
218,Garbag Bags - Black,1
219,General Purpose Trigger,1
220,Grenadine,1
221,Grouper - Fresh,1


In [169]:
merge = pd.merge(rank_products, pd.DataFrame(prod_cust_pivot1['new_customer']), on='ProductName')
merge.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merge[merge['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
1,Knife Plastic - White,4,0.0
3,Muffin - Carrot Individual Wrap,3,0.0
4,Beef - Short Loin,3,0.0
6,Chips Potato Salt Vinegar 43g,3,0.0
7,Tea - Jasmin Green,3,0.0


### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [170]:
product_dist1 = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot1, 'euclidean'))),
                         index=prod_cust_pivot1.index, columns=prod_cust_pivot1.index)
product_dist1

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.003559,0.003927,0.004175,0.004102,0.003616,0.003767,0.003876,0.004024,0.004023,...,0.003794,0.003749,0.003846,0.004037,0.003839,0.004026,0.003908,0.003896,0.003860,0.003578
"Appetizer - Mini Egg Roll, Shrimp",0.003559,1.000000,0.003670,0.003697,0.003692,0.003413,0.003479,0.003900,0.003789,0.003486,...,0.003639,0.003481,0.003624,0.003751,0.003639,0.003508,0.003691,0.003749,0.003564,0.003475
Appetizer - Mushroom Tart,0.003927,0.003670,1.000000,0.004234,0.003849,0.003789,0.003819,0.004120,0.004110,0.003686,...,0.003920,0.003683,0.003812,0.004418,0.004155,0.004111,0.004095,0.004060,0.003937,0.003698
Appetizer - Sausage Rolls,0.004175,0.003697,0.004234,1.000000,0.004078,0.003974,0.003808,0.004169,0.003998,0.003837,...,0.003922,0.003853,0.003962,0.004551,0.004089,0.003872,0.004082,0.004020,0.003878,0.003718
Apricots - Dried,0.004102,0.003692,0.003849,0.004078,1.000000,0.003755,0.003722,0.004073,0.003895,0.003778,...,0.003760,0.003669,0.003869,0.004233,0.003836,0.003718,0.004018,0.003955,0.003811,0.003767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.004026,0.003508,0.004111,0.003872,0.003718,0.003727,0.003720,0.004040,0.003987,0.003776,...,0.004007,0.003635,0.003694,0.004068,0.003900,1.000000,0.003849,0.003791,0.003749,0.003608
Yoghurt Tubes,0.003908,0.003691,0.004095,0.004082,0.004018,0.003738,0.003923,0.004075,0.003956,0.003794,...,0.003932,0.003769,0.003846,0.004098,0.004100,0.003849,1.000000,0.003876,0.003834,0.003691
"Yogurt - Blueberry, 175 Gr",0.003896,0.003749,0.004060,0.004020,0.003955,0.003872,0.003787,0.004037,0.004051,0.003695,...,0.003866,0.003758,0.003844,0.004109,0.003874,0.003791,0.003876,1.000000,0.003772,0.003714


In [171]:
prod=prod_cust_pivot1['new_customer'].reset_index()
products=list(prod[prod['new_customer']!=0]['ProductName'])
similar_prod=[]
for p in products: 
    similar_prod.append(list(product_dist[p].sort_values(ascending=False)[1:].head().index))
flat_list = [item for sublist in similar_prod for item in sublist]
for a in flat_list:
    if a in products:
        flat_list.remove(a) 
times={}
for f in flat_list:
    times[f]=times.get(f,0)+1
k = Counter(times)
high = k.most_common(5)
top5=[]
for h in high:
    top5.append(h[0]) 

top5

['Beef - Chuck, Boneless',
 'Towels - Paper / Kraft',
 'Rice - Jasmine Sented',
 'Initation Crab Meat',
 'V8 - Berry Blend']